# Working with child classes

There can be scenarios where it makes sense to subclass different methods for a single stage.
E.g. testing out dense neural network vs. a CNN which are both defined in a custom class.

Such a scenario can be handled by using a class for each method, which in the following are referred to as child classes although they do not necessarily have to inherit from a parent class.

In [5]:
from pytrack import PyTrackProject, config

config.nb_name = "PassingClasses.ipynb"
project = PyTrackProject()
project.create_dvc_repository()

2021-10-19 10:41:21,669 (INFO): Setting up GIT/DVC repository.


In the following we define two operations that have different arguments and perform different calculations.
We apply the `check_signature` decorator, which asserts the signature names with the class attributes. I.e. if the signature has an argument `factor` the associated class attribute has to be called `self.factor`.

We then collect all available operations in a dictionary in the PyTrack stage and use `inspect` and `getattr` to store the parameters.
In the `run` method we then instatiate a new version of the operation doing the actual computation.
Again, this is required because it will be excecuted in a new Python kernel.

In [6]:
from pytrack import PyTrack, DVC
from pytrack.utils.decorators import check_signature
import inspect

class Operation1:
    """
    An implementation or algorithm you want to 
    be tracked by py-track.
    """
    @check_signature
    def __init__(self, factor):
        self.factor = factor

    def convert_input(self, inp):
        return inp * self.factor

class Operation2:
    """
    Another implementation or algorithm you want to 
    be tracked by py-track. should accomplish the same
    goal as Operation1 but perhaps in a different way.
    """
    @check_signature
    def __init__(self, factor, shift):
        self.factor = factor
        self.shift = shift

    def convert_input(self, inp):
        return inp * self.factor + self.shift


@PyTrack()
class ChildHandler:
    """
    PyTrack enabled class which handles calling the
    implementations.
    """
    
    methods = {
        Operation1.__name__: Operation1,
        Operation2.__name__: Operation2
    }

    operation = DVC.params()
    operation_parameter = DVC.params()
    inputs = DVC.params()
    output = DVC.result()

    def __call__(self, operation: object, inp):
        """
        This is what will be called when you run the
        script. For example:
            data = ChildHandler(Operation1(), ...)
        Calling of this class will add it to the 
        computation graph which is later executed.
        """
        self.operation = operation.__class__.__name__
        operation_parameter = {}
        for key in inspect.signature(operation.__class__.__init__).parameters:
            if key == "self":
                continue
            operation_parameter[key] = getattr(operation, key)
        self.operation_parameter = operation_parameter
        # Note we have to use "=" here and can not update it iteratively!

        self.inputs = inp
    def run(self):
        """
        Called on the backed to execute the methods in
        one of the child classes. In this case, the 
        method we want to run in the child class is 
        called convert_input.
        """
        Operation = self.methods[self.operation]
        operation = Operation(**self.operation_parameter)

        self.output = operation.convert_input(self.inputs)

2021-10-19 10:41:24,218 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/py-track.


[NbConvertApp] Converting notebook PassingClasses.ipynb to script
[NbConvertApp] Writing 5086 bytes to PassingClasses.py


We are using `inspect.signature` and assume that the class attributes have the same name as the `__init__`
 parameters.
 They can be changed in any way though.

Let us now use both passed child methods and see how it works out.

In [7]:
child_handler = ChildHandler()
operation = Operation1(factor=3)
child_handler(operation=operation, inp=15)

project.queue(name="Op1")

child_handler = ChildHandler()
operation = Operation2(factor=2, shift=500)
child_handler(operation=operation, inp=15)

project.queue(name="Op2")
project.run_all()

2021-10-19 10:41:27,410 (WARNING): --- Writing new DVC file! ---
2021-10-19 10:41:27,411 (WARNING): Overwriting existing configuration!
2021-10-19 10:41:30,885 (INFO): Creating 'dvc.yaml'
Adding stage 'ChildHandler' in 'dvc.yaml'

To track the changes with git, run:

	git add outs/.gitignore dvc.yaml

2021-10-19 10:41:30,890 (INFO): Running git add
2021-10-19 10:41:30,993 (INFO): Queue DVC stage
Queued experiment '1c37fdd' for future execution.
2021-10-19 10:41:33,707 (WARNING): --- Writing new DVC file! ---
2021-10-19 10:41:33,708 (WARNING): Overwriting existing configuration!
2021-10-19 10:41:36,742 (INFO): Modifying stage 'ChildHandler' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml

2021-10-19 10:41:36,748 (INFO): Running git add
2021-10-19 10:41:36,810 (INFO): Queue DVC stage
Queued experiment '4ba3132' for future execution.
2021-10-19 10:41:39,204 (INFO): RUN DVC stage
Running stage 'ChildHandler':
> python3 -c "from src.ChildHandler import ChildHandler; Ch

	outs/.gitignore, src/__pycache__/ChildHandler.cpython-39.pyc


Running stage 'ChildHandler':
> python3 -c "from src.ChildHandler import ChildHandler; ChildHandler(load=True).run()"
2021-10-19 10:41:45,907 (WARNING): No results found!
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'


	outs/.gitignore, src/__pycache__/ChildHandler.cpython-39.pyc



Reproduced experiment(s): Op2, Op1
To apply the results of an experiment to your workspace run:

	dvc exp apply <exp>

To promote an experiment to a Git branch run:

	dvc exp branch <exp> <branch>

2021-10-19 10:41:47,267 (INFO): Running git add


We can now load them and see the results

In [10]:
project.load("Op1")
op1_rev = ChildHandler(load=True)
project.load("Op2")
op2_rev = ChildHandler(load=True)
print(f"Op1: {op1_rev.operation_parameter} \t {op1_rev.output}")
print(f"Op2: {op2_rev.operation_parameter} \t {op2_rev.output}")

Changes for experiment 'Op1' have been applied to your current workspace.
Changes for experiment 'Op2' have been applied to your current workspace.
Op1: {'factor': 3} 	 45
Op2: {'factor': 2, 'shift': 500} 	 530


## Parent classes

This nested methods can get arbitrarily complicated, having different dependencies and outputs.
In that case it might be useful to have a common parent class and loop over e.g., a list of dependecies as such:

In [18]:
class OperationBase:
    def __init__(self):
        self.dependencies = []
        self.outs = []

class Operation3(OperationBase):
    def __init__(self):
        super().__init__()
        self.dependencies = ['File1', 'File2', 'File3']
        self.outs = ['Out1.txt', 'Out2.txt']

@PyTrack()
class ChildHandlerWithBase:
    methods = {
            Operation3.__name__: Operation3
        }
    method = DVC.params()
    deps = DVC.deps()
    outs = DVC.outs()       

    def __call__(self, operation: OperationBase):
        self.deps = [x for x in operation.dependencies]
        self.outs = [x for x in operation.outs]

    def run(self):
        method = self.methods[self.method]

2021-10-19 13:18:31,902 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/py-track.


[NbConvertApp] Converting notebook PassingClasses.ipynb to script
[NbConvertApp] Writing 5743 bytes to PassingClasses.py


Now we can pass our custom operation with a list of dependencies and outputs

In [19]:
child_handler = ChildHandlerWithBase()
operation = Operation3()
child_handler(operation)

2021-10-19 13:18:36,145 (WARNING): --- Writing new DVC file! ---
2021-10-19 13:18:36,146 (WARNING): Overwriting existing configuration!
2021-10-19 13:18:39,994 (INFO): Modifying stage 'ChildHandlerWithBase' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml



If we now look at the PyTrack dependencies and outputs they are all set correctly.

In [20]:
print(ChildHandlerWithBase(load=True).deps)
print(ChildHandlerWithBase(load=True).outs)

2021-10-19 13:18:40,012 (WARNING): No results found!
['File1', 'File2', 'File3']
2021-10-19 13:18:40,020 (WARNING): No results found!
['Out1.txt', 'Out2.txt']
